<a href="https://colab.research.google.com/github/wikiban43/Misinformation-Analysis/blob/main/community.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import os
import json
import time
import pandas as pd
from datetime import datetime,timedelta
import time
import pickle
from google.colab import drive
drive.mount('/content/drive')
import tweepy
#consumer_key = "cDiClkP49iTsbc3G7Ti5VJcLp"
#consumer_secret = "2d4sR9ypcH7bNYFrDN6qeHf3NpUGBfOwBwPK4kS3JGRwQTuD5B"
#Bearer = AAAAAAAAAAAAAAAAAAAAAD%2BcYgEAAAAAHw4G9h4lbyrz0Um6UPwmQuXMegI%3DMK7BwPEtDWySjj9XQnKxBZ5ek5BOurIy73soU7Ir2TxMo5TdO9
#access_token = "2934858343-MoyJzYCbfgK7u356NwOOxRh9VShk2J8jCwkQTzK"
#access_secret = "CW5gOfLeEQHs4El5UarVe3MGvDjMjXoTPYnn2vjXQIwN0"

# Authenticate Tweepy connection to Twitter API
#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_secret)
#api = tweepy.API(auth, wait_on_rate_limit=True)
#path = "drive/My Drive/MSR/ssr_keywords3/final tweets/"

# Community Analysis

In [ ]:
#READ DATA FOR THE EVENTS

path = "drive/My Drive/MSR/icwsm_datasets/"
df = pd.read_csv(path+"caa_nrc.csv")
#df = pd.read_csv(path+"budget2022_jan_jul.csv")
#df = pd.read_csv(path+"budget2022_6months.csv")


#Some preprocessing for budget data
#path = "drive/My Drive/MSR/"#icwsm_datasets/old datasets/"
#df1 = pd.read_csv(path+'budget2022_jan_mar.csv')
#df2 = pd.read_csv(path+'budget2022_mar_jul.csv')
#df = pd.concat([df1,df2])

df.shape  

(104533, 22)

In [ ]:
df = df_champs

In [ ]:
#Get retweet count received by each user (without any duplicate filtering)
df_rt = df.groupby(["author.username"])["public_metrics.retweet_count"].sum().reset_index()
df_rt = df_rt.sort_values(by=["public_metrics.retweet_count"],ascending=False).reset_index()
#stats_df = get_cdf_data(df_rt,"public_metrics.retweet_count")
#stats_df.tail()
#import plotly.express as px
#fig = px.line(stats_df,x="public_metrics.retweet_count",y="cdf")#,x="author.username")
#fig.show()

#SELECT USERS WITH HIGH RETWEET ATTRACTION FROM THE DATAFRAME, BASED ON THE CDF
df_rt_high_retweets = df_rt.loc[df_rt["public_metrics.retweet_count"]>2800].reset_index()
high_retweet_users = list(set(list(df_rt_high_retweets["author.username"].unique())))
print(high_retweet_users)

['advkrishnakantp', 'ajitchauhaan', 'MisaBharti', 'ojha_ivar', 'ganpats9', 'Aryanyadav17', 'iamNinJaBrother', 'raamhanuman', 'anuragd11883942', 'Huda12_97', 'satydev33', 'krunal_sardhara', 'OmprakashAwad12', 'kanaram190', 'ManojBajpaiBBK', 'AshokKReddy', 'madangzp', 'DKJAIN_17', 'RahulPa96882581', 'mrsinghCricUmp', 'VijayK78156267', 'ahmed_sf', 'amitkah19944828', 'theuncleeinstin', 'RaviKrMandalBJP', 'KingKha41353185', 'Neelakandan_22', 'akhileshnag', 'nawneetpandey45', 'top_trending_gk', 'gauravt84821873', 'veerkumars', 'thecivegg', 'khizarhashmi119', 'Ashutos05194354', 'peetamsingh1976', 'rahultiwari_jbp', 'RitvikPatel1', 'drbmishra7', 'byDeepak', 'sbjt7777', 'MrHarishKumar', 'shuklabn75', 'ARvindJ37717113', 'VinayakH2016', 'Kanhaiya1101', 'AwaraRaahi', 'JyotPurohit', 'jai_mahakal_g', 'GoanSlayer', 'Sangachhwadham', 'lazimdekhenge', 'ps_rathore7', 'Kaavi2021', 'virendrajain48', 'its_singhaman', 'Sharma08162548', 'RAJKUMA06648417', 'NandeshSwami', 'Khediwal', 'rajputparas1', 'Ranvijay

In [ ]:
from datetime import datetime
import ast

#HELPER METHODS TO GET THE DATE AND AUTHOR NAME FROM TEXT
def get_date(string):
  try:
    date = datetime.strptime(string, '%Y-%m-%dT%H:%M:%S.%fZ')
  except:
    date = string
    return date
  #date = str(date.month)+"/"+str(date.year)
  return date.date()

def get_author_name(string):
  try:
    d = ast.literal_eval(string)
    return d["username"]
  except:
    return "None"

df["created_at"] = df["created_at"].apply(lambda x: get_date(x))
df["author.username"] = df["author"].apply(lambda x: get_author_name(x))  #For Budget only

In [ ]:
#GET MEDIA AND INFLUENCERS FROM THE DISMIS DATASET

df_dismis = pd.read_excel("drive/My Drive/MSR/icwsm_analysis/influencers_final.xlsx")
df_medinf = df_dismis.loc[(df_dismis["sub_category"]=='media_house') | (df_dismis["sub_category"]=='platform_first')].reset_index()

df_dismis["sub_category"].unique()

array(['entertainer', 'organisation', 'sportsperson', 'media_house',
       'team', 'person', 'journalist', 'police', 'official',
       'informational', 'religious organisation', 'brand', 'academic',
       'specialist', 'social worker', 'fan account', 'platform_first',
       'writer', 'humor', 'bureaucrat', 'defence',
       'research organisation', 'law & policy', 'designer'], dtype=object)

In [ ]:
#Politicians list
df_politicians = pd.read_csv('drive/My Drive/MSR/icwsm_analysis/politicians.csv')
pol_l = list(df_politicians['screen_name'].unique())
#Media and influencer list
media_inf_l = list(df_medinf["username"].unique())

#df with media and influencers
#df_champs = df.loc[df["author.username"].isin(media_inf_l)].reset_index()

#df with high retweet receivers and media and influencers
#df_champs = df.loc[(df["author.username"].isin(high_retweet_users))|(df["author.username"].isin(media_inf_l))].reset_index()

#df with high retweet receivers, politicians, and media and influencers
#df_champs = df.loc[(df["author.username"].isin(high_retweet_users))|(df["author.username"].isin(media_inf_l))|(df["author.username"].isin(pol_l))].reset_index()

#df with politicians, and media and influencers
df_champs = df.loc[(df["author.username"].isin(media_inf_l))|(df["author.username"].isin(pol_l))].reset_index()




In [ ]:
print("Media and influencers:"+str(len(media_inf_l)))
#print("Highly retweeted users:"+str(len(high_retweet_users)))
print("Final dataset with both has users:"+str(df_champs["author.username"].nunique()))
df_champs.shape

Media and influencers:1411
Final dataset with both has users:3860


(42685, 13)

In [ ]:
#df_champs.shape
#df_champs.drop_duplicates().shape
#df_champs = df_champs[["id","text","created_at","author.username","public_metrics.retweet_count","referenced_tweets"]].drop_duplicates()
#df_champs = df_champs[["id","text","created_at","author.username","public_metrics.retweet_count","referenced_tweets"]].drop_duplicates()
df_champs = df_champs[["id","text","created_at","author.username","referenced_tweets"]].drop_duplicates()

In [ ]:
#df_media = df_medinf2.loc[df_medinf2["sub_category"]=="media_house"].reset_index() #Enable if only media tweets are needed
#df_inf = df_medinf2.loc[df_medinf2["sub_category"]=="platform_first"].reset_index()#Enable if only influencer tweets are needed
df = df_champs

In [ ]:
#SOME PREPROCESSING ON THE DF TO GET THE DATES IN STRING FORMAT ETC
df = df.sort_values(by="created_at")#.reset_index()
df["created_at"] = df["created_at"].astype(str)
df.reset_index(drop=True,inplace=True)

In [ ]:
#METHOD TO GET RETWEET NETWORK FROM ANY DF
def get_network(df):
  username_l = list(df["author.username"].unique())
  #Get the retweeted users
  if "text" in df.columns:
    df["retweeted user"] = df["text"].apply(lambda x: get_retweeted_user(x))

  #Get all retweet edges using Soham's data (A,B would mean A retweets B)
  #edges = []
  df['edges'] = list(zip(df["author.username"], df["retweeted user"]))
  #Unique edges (unweighted graph)
  edges = df['edges']

  #Get only the within group edges for network analysis
  relevant_edges = []
  for edge in edges:
    if edge[0] in username_l and edge[1] in username_l:
      relevant_edges.append(edge)
  edges = relevant_edges

  #Weighted graph if needed
  #weight_dict = get_edges_with_weights(df['edges'].to_list())
  #dump_csv_for_community_detection(weight_dict)

  #Remove duplicate edges for unweighted graph
  edges = set(edges)
  print("Number of unique edges: "+str(len(edges)))
  print("\n")
  if len(edges) == 0:
    print("No retweet edges found!")
    return None
  #Get the RT network between core users (one edge only between a pair of users even if there are multiple RTs between them)
  # generate generic network graph instance
  nx_graph = nx.DiGraph()
  for edge in edges:#[0:1000]:
    #print(edges_dict[edge])
    try:
      nx_graph.add_edge(edge[0],edge[1])
    except:
      continue
  return nx_graph

In [ ]:
#GET THE RT NETWORK FOR THIS EVENT

from matplotlib import pyplot as plt
import networkx as nx
import networkx.algorithms.community as nxcom

#pos = nx.spring_layout(nx_graph, k=0.1)
#plt.rcParams.update({'figure.figsize': (15, 10)})
#nx.draw_networkx(nx_graph,pos=pos,node_size=0,edge_color="#444444",alpha=0.05,with_labels=False)
nx_graph = get_network(df)

#GET COMMUNITIES USING THE GREEDY MODULARITY TECHNIQUE FOR THE RETWEET GRAPH FOR THIS EVENT
communities = sorted(nxcom.greedy_modularity_communities(nx_graph), key=len, reverse=True)
len(communities)

Number of unique edges: 4948




158

In [ ]:
pickle.dump(communities,open('drive/My Drive/MSR/icwsm_analysis/Budget_communities_medinf_pol.pickle','wb'))
#caa_comm = pickle.load(open('drive/My Drive/MSR/icwsm_analysis/CAA_communities.pickle','rb'))

In [ ]:
df.loc[df["author.username"]=='BhootSantosh']

In [ ]:
caa_comm = communities

In [ ]:
import networkx as nx
#FOR EACH COMMUNITY, FIND THE CENTRAL/MOST INFLUENTIAL USER
for i in range(0,len(caa_comm)):
  iterator=iter(caa_comm[i])
  comm = list(iterator)
  df_comm = df.loc[df["author.username"].isin(comm)].reset_index()
  comm_G = get_network(df_comm)
  influence_dict = nx.pagerank_scipy(comm_G)
  leader = max(influence_dict, key=influence_dict.get)
  print("The leader of community-"+str(i)+" is:"+leader+" with PageRank "+str(influence_dict[leader]))
#for username in iterator:
#  print(username)


In [ ]:
 #VISUALIZE GRAPH BASED ON COMMUNITIES (NOT IMMEDIATELY NEEDED)
 
 def set_node_community(G, communities):
  '''Add community to node attributes'''
  for c, v_c in enumerate(communities):
      for v in v_c:
          # Add 1 to save 0 for external edges
          G.nodes[v]['community'] = c + 1
def set_edge_community(G):
  '''Find internal edges and add their community to their attributes'''
  for v, w, in G.edges:
      if G.nodes[v]['community'] == G.nodes[w]['community']:
          # Internal edge, mark with community
          G.edges[v, w]['community'] = G.nodes[v]['community']
      else:
          # External edge, mark as 0
          G.edges[v, w]['community'] = 0
def get_color(i, r_off=1, g_off=1, b_off=1):
  '''Assign a color to a vertex.'''
  r0, g0, b0 = 0, 0, 0
  n = 16
  low, high = 0.1, 0.9
  span = high - low
  r = low + span * (((i + r_off) * 3) % n) / (n - 1)
  g = low + span * (((i + g_off) * 5) % n) / (n - 1)
  b = low + span * (((i + b_off) * 7) % n) / (n - 1)
  return (r, g, b) 

G_social = nx_graph

plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'figure.figsize': (15, 10)})
plt.style.use('dark_background')
# Set node and edge communities
set_node_community(G_social, communities)
set_edge_community(G_social)
# Set community color for internal edges
external = [(v, w) for v, w in G_social.edges if G_social.edges[v, w]['community'] == 0]
internal = [(v, w) for v, w in G_social.edges if G_social.edges[v, w]['community'] > 0]
internal_color = ["black" for e in internal]
node_color = [get_color(G_social.nodes[v]['community']) for v in G_social.nodes]
# external edges
nx.draw_networkx(
    G_social,
    pos=pos,
    node_size=0,
    edgelist=external,
    edge_color="silver",
    node_color=node_color,
    alpha=0.2,
    with_labels=False)
# internal edges
nx.draw_networkx(
    G_social, pos=pos,
    edgelist=internal,
    edge_color=internal_color,
    node_color=node_color,
    alpha=0.05,
    with_labels=False)